In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import Document
from langchain.vectorstores import FAISS 
from langchain.chat_models import init_chat_model
from langchain.schema.runnable import RunnableMap
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [2]:
### Custom Semantic Chunker With Threshold
class ThresholdSematicChunker:
    # If model does not exists, you can just add name of model in model_name, it will download automatically
    def __init__(self,model_name="D:\\model\\BAAI-bge-m3",threshold=0.6):
        self.model=SentenceTransformer(model_name)
        self.threshold=threshold 

    def split(self, text: str):
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        embeddings = self.model.encode(sentences)
        chunks = []
        current_chunk = [sentences[0]]

        for i in range(1, len(sentences)):
            sim = cosine_similarity([embeddings[i - 1]], [embeddings[i]])[0][0]
            if sim >= self.threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(". ".join(current_chunk) + ".")
                current_chunk = [sentences[i]]

        chunks.append(". ".join(current_chunk) + ".")
        return chunks
    
    def split_documents(self,docs):
        result=[]
        for doc in docs:
            for chunk in self.split(doc.page_content):
                result.append(Document(page_content=chunk, metadata=doc.metadata))

        return result

    

In [3]:
# Sample text
sample_text = """
    ວິທະຍາສາດຄອມພິວເຕີແມ່ນສາຂາວິຊາທີ່ສຳຄັນໃນຍຸກປັດຈຸບັນ.
    ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ.
    ນັກສຶກສາສາມາດຫາວຽກເຮັດງານທຳໄດ້ງ່າຍຂຶ້ນຖ້າມີຄວາມຮູ້ດ້ານນີ້.
    ປັນຍາປະດິດແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ.
    AI ສາມາດນຳມາໃຊ້ໃນຫຼາຍຂະແໜງການ.
    ຫຼາຍບໍລິສັດໃຫຍ່ກຳລັງລົງທຶນໃນການພັດທະນາ AI. 
    ສປປ ລາວແມ່ນປະເທດທີ່ມີວັດທະນະທຳອັນຍາວນານ.
    ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ.
    ຄົນລາວຄົນຈິງໃຈ
"""

doc = Document(page_content=sample_text)
doc

Document(metadata={}, page_content='\n    ວິທະຍາສາດຄອມພິວເຕີແມ່ນສາຂາວິຊາທີ່ສຳຄັນໃນຍຸກປັດຈຸບັນ.\n    ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ.\n    ນັກສຶກສາສາມາດຫາວຽກເຮັດງານທຳໄດ້ງ່າຍຂຶ້ນຖ້າມີຄວາມຮູ້ດ້ານນີ້.\n    ປັນຍາປະດິດແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ.\n    AI ສາມາດນຳມາໃຊ້ໃນຫຼາຍຂະແໜງການ.\n    ຫຼາຍບໍລິສັດໃຫຍ່ກຳລັງລົງທຶນໃນການພັດທະນາ AI. \n    ສປປ ລາວແມ່ນປະເທດທີ່ມີວັດທະນະທຳອັນຍາວນານ.\n    ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ.\n    ຄົນລາວຄົນຈິງໃຈ\n')

In [4]:
### Chunking
chunker=ThresholdSematicChunker(threshold=0.6)
chunks=chunker.split_documents([doc])
chunks

[Document(metadata={}, page_content='ວິທະຍາສາດຄອມພິວເຕີແມ່ນສາຂາວິຊາທີ່ສຳຄັນໃນຍຸກປັດຈຸບັນ. ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ. ນັກສຶກສາສາມາດຫາວຽກເຮັດງານທຳໄດ້ງ່າຍຂຶ້ນຖ້າມີຄວາມຮູ້ດ້ານນີ້.'),
 Document(metadata={}, page_content='ປັນຍາປະດິດແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ.'),
 Document(metadata={}, page_content='AI ສາມາດນຳມາໃຊ້ໃນຫຼາຍຂະແໜງການ. ຫຼາຍບໍລິສັດໃຫຍ່ກຳລັງລົງທຶນໃນການພັດທະນາ AI.'),
 Document(metadata={}, page_content='ສປປ ລາວແມ່ນປະເທດທີ່ມີວັດທະນະທຳອັນຍາວນານ.'),
 Document(metadata={}, page_content='ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ. ຄົນລາວຄົນຈິງໃຈ.')]

In [5]:
### VectorStore 
from langchain_huggingface import HuggingFaceEmbeddings

## Initialize a simple Embedding model(no API Key needed!)
embeddings=HuggingFaceEmbeddings(
    model_name="D:\\model\\BAAI-bge-m3"
) 
vectorstore=FAISS.from_documents(chunks,embeddings)
retriever=vectorstore.as_retriever()



In [6]:
template = """Answer the question based on the following context:
Important: response in lao language only
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\nImportant: response in lao language only\n{context}\n\nQuestion: {question}\n')

In [10]:
## LLM
llm=init_chat_model(model="groq:meta-llama/llama-4-maverick-17b-128e-instruct",temperature=0.4)

### LCEL Chain With retrieval

rag_chain=(
    RunnableMap(
        {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],  
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

query = {"question": "ປະຊາຊົນລາວເປັນຄົນໃຈດີບໍ່?"}
result = rag_chain.invoke(query)

print("ຄຳຕອບທີ່ 1 :", result)

query2 = {"question": "ວິທະຍາສາດຄອມພິວເຕີສຳຄັນບໍ່"}
result2 = rag_chain.invoke(query2)

print("ຄຳຕອບທີ່ 2 :",result2)

ຄຳຕອບທີ່ 1 : ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ, ຄົນລາວຄົນຈິງໃຈ. ດັ່ງນັ້ນ, ປະຊາຊົນລາວເປັນຄົນໃຈດີ.
ຄຳຕອບທີ່ 2 : ວິທະຍາສາດຄອມພິວເຕີແມ່ນສາຂາວິຊາທີ່ສຳຄັນໃນຍຸກປັດຈຸບັນ. ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ ແລະ ສາມາດຫາວຽກເຮັດງານທຳໄດ້ງ່າຍຂຶ້ນ. ດັ່ງນັ້ນ, ວິທະຍາສາດຄອມພິວເຕີຈຶ່ງສຳຄັນ.
